In [7]:
using NearestNeighbors
using BenchmarkTools

Let us create some data

In [8]:
data = rand(Float32, 128, 10^6)

128×1000000 Matrix{Float32}:
 0.687857   0.785659   0.676999    …  0.523741    0.809215    0.295437
 0.149933   0.339917   0.816843       0.00970232  0.757438    0.243993
 0.658733   0.385988   0.43249        0.378       0.419239    0.139605
 0.459716   0.195461   0.177259       0.287488    0.344925    0.282759
 0.224169   0.814845   0.42296        0.51748     0.775577    0.601469
 0.899717   0.144047   0.240218    …  0.344248    0.00593603  0.136317
 0.341651   0.245336   0.74273        0.442731    0.461624    0.15544
 0.557462   0.570779   0.715726       0.947273    0.353633    0.807687
 0.339329   0.681877   0.776807       0.402773    0.865587    0.743952
 0.356522   0.502739   0.206005       0.831999    0.118927    0.98116
 0.549483   0.729211   0.641927    …  0.145109    0.171613    0.22724
 0.598691   0.332117   0.60057        0.887142    0.328095    0.986986
 0.394814   0.402753   0.939232       0.508874    0.558687    0.34845
 ⋮                                 ⋱                

Then we can instanciate a tree with the data and use it for searching vectors to a query vector

In [9]:
brutetree = BruteTree(data)

BruteTree{StaticArraysCore.SVector{128, Float32}, Euclidean}
  Number of points: 1000000
  Dimensions: 128
  Metric: Euclidean(0.0)
  Reordered: false

Let us consider a query vector

In [10]:
query = data[:,4];

The method **`knn`** returns the top `k` items to the query

In [13]:
k = 10
idx, distances = knn(brutetree, query, k) 

([368682, 593392, 446008, 8071, 512932, 777413, 763039, 163584, 525605, 4], Float32[3.702029, 3.6918645, 3.6857388, 3.660195, 3.674085, 3.6798215, 3.6688938, 3.6510324, 3.5827184, 0.0])

In [29]:
@btime knn($brutetree, $query, 1) 

  24.324 ms (2 allocations: 128 bytes)


([4], Float32[0.0])

In [31]:
@benchmark idx, distances = knn(brutetree, query, k) 

BenchmarkTools.Trial: 204 samples with 1 evaluation.
 Range (min … max):  24.327 ms …  26.432 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     24.498 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   24.589 ms ± 280.331 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    █▄▇▆▁ ▃                                                     
  ▆▅███████▄▇▆▇▅▇▅▄▃▅▃▄▄▄▅▃▅▃▃▁▃▁▁▁▁▁▃▁▁▁▁▁▃▃▃▁▁▁▁▃▁▁▁▁▁▁▃▁▁▁▃ ▃
  24.3 ms         Histogram: frequency by time         25.7 ms <

 Memory estimate: 272 bytes, allocs estimate: 3.

In [15]:
query_batch = data[:,10:20];

In [8]:
@benchmark idx, distances = knn(brutetree, query_batch, 1) 

BenchmarkTools.Trial: 19 samples with 1 evaluation.
 Range (min … max):  263.135 ms … 267.479 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     265.197 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   265.090 ms ±   1.074 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

                             ▃      █▃    ▃                      
  ▇▁▇▁▁▁▁▁▁▇▇▁▁▁▁▇▁▁▇▁▁▁▁▁▁▇▁█▇▁▁▁▁▁██▁▁▁▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▁
  263 ms           Histogram: frequency by time          267 ms <

 Memory estimate: 7.34 KiB, allocs estimate: 27.

The query can also be a batch of vectors

In [17]:
idx, distances = knn(brutetree, query_batch, 1) 

([[10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20]], Vector{Float32}[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]])

Here we can see that vectors from the query batch have the closest matches in indices 10 to 20 (which are the indices we used to select them!)

In [10]:
idx

11-element Vector{Vector{Int64}}:
 [10]
 [11]
 [12]
 [13]
 [14]
 [15]
 [16]
 [17]
 [18]
 [19]
 [20]

Note that creating a `BruteTree` instance can take some time 

In [11]:
@benchmark brutetree = BruteTree(data)

BenchmarkTools.Trial: 55 samples with 1 evaluation.
 Range (min … max):  90.558 ms …  94.109 ms  ┊ GC (min … max): 0.00% … 0.43%
 Time  (median):     91.591 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   91.803 ms ± 721.254 μs  ┊ GC (mean ± σ):  0.15% ± 0.21%

              █ ▄    ▂                                          
  ▄▁▁▁▄▄▁▁▄▄▁▁█▄█▆▄▆██▁▄█▁▆▄█▁▁▄▄▆▁▄▁▁▄▆▆▁▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▄ ▁
  90.6 ms         Histogram: frequency by time         93.8 ms <

 Memory estimate: 488.28 MiB, allocs estimate: 5.

Note that even id data is not store the time is similar

In [12]:
@benchmark brutetree2 = BruteTree(data; storedata=false)

BenchmarkTools.Trial: 55 samples with 1 evaluation.
 Range (min … max):  90.483 ms …  93.861 ms  ┊ GC (min … max): 0.00% … 0.43%
 Time  (median):     91.286 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   91.537 ms ± 857.910 μs  ┊ GC (mean ± σ):  0.22% ± 0.22%

  ▁      ▃ ▁█       ▁▁ ▁  ▁                                     
  █▇▁▄▁▄▄█▇██▇▁▄▇▁▄▁██▁█▇▇█▁▁▄▁▄▄▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▄▁▁▇▁▁▁▁▁▁▇ ▁
  90.5 ms         Histogram: frequency by time         93.8 ms <

 Memory estimate: 488.28 MiB, allocs estimate: 6.

If we want to decrease the constructiontime of the Brute tree we can use a single datapoint

In [20]:
aux = data[:,1];
@benchmark brutetree2 = BruteTree(aux;storedata=false)

BenchmarkTools.Trial: 10000 samples with 136 evaluations.
 Range (min … max):  705.272 ns … 102.496 μs  ┊ GC (min … max): 0.00% … 98.90%
 Time  (median):     724.265 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   771.833 ns ±   1.020 μs  ┊ GC (mean ± σ):  1.31% ±  0.99%

  ▄▇█▇▆▄▅▅▅▄▃▂▂▂▂▁▂▂▂▂▂▂▂▁▂▁▁▁ ▁          ▂▂▃▃▃▃▂▃▂▂▂▁          ▂
  ███████████████████████████████▇█▇▇▆▅▇██████████████████▇▆▇▅▅ █
  705 ns        Histogram: log(frequency) by time        954 ns <

 Memory estimate: 752 bytes, allocs estimate: 7.

In [21]:
idx, distances = knn(brutetree2, data, query, 1) 

LoadError: UndefVarError: brutetree2 not defined